In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [ ]:
! gdown --id '1EvyZ1gwCRjS0yk29ePnbW4nDr_8LhFl9' --output ed_base.jpg

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1EvyZ1gwCRjS0yk29ePnbW4nDr_8LhFl9
To: /content/ed_base.jpg
100% 77.0k/77.0k [00:00<00:00, 65.4MB/s]


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00


In [ ]:
# modules import
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch import nn

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import sys
import numpy as np
import torch
from tqdm.notebook import tqdm
from torch.optim import Adam
import os
import subprocess

from matplotlib import pyplot as plt
from datetime import datetime
import shutil
import random
import itertools
import evaluate

bleu_metric = evaluate.load("bleu")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
VALIDATION_PROP = 0.1
RANDOM_STATE = 79

X_TRAIN_PATH = 'en_train.txt'
Y_TRAIN_PATH = 'ru_train.txt'
X_VALID_PATH = 'en_valid.txt'
Y_VALID_PATH = 'ru_valid.txt'
TRAIN = False

HIDDEN_SIZE = 256
LSTM_LAYERS = 3
INPUT_SIZE = 256
BATCH_SIZE = 32
IS_BIDIRECTIONAL_LAYERS = False
CNT_CLASSES = None

SOS_TOKEN = "<SOS>" # start of sentence token
EOS_TOKEN = "<EOS>" # end of sentence token
UNK_TOKEN = "<UNK>" # unknown token
PAD_TOKEN = "<PAD>" # padding token
PAD_IND = 3
SOS_IND = 0
EOS_IND = 1
EPOCHS = 10
MODEL_NAME = 'lstm_basic_model'
SCHEDULER_LAMBDA_PARAM = 0.95
device = 'cuda' if torch.cuda.is_available() else 'cpu'

MODEL_FOLDER_PATH = MODEL_NAME
LR = 0.0004
CLIP = 1
TEACHER_FORCING_RATIO = 0
FULL_DATA_PATH = 'data.txt'
DOWNLOAD_DATA_URL = 'https://www.dropbox.com/s/yy2zqh34dyhv07i/data.txt?dl=1'

WRITE_CNT_STEPS = 100
SCHEDULER_CNT_STEPS = 300
GREEDY_DECODING_LABEL = 'greedy'
BEAM_SEARCH_DECODING_LABEL = 'beam_search'

NEC_TOKENS_LEN = 100  # tokens max length for inference
MAX_TEXT_LEN = 100 # max symbols length for text
BEAM_WIDTH = 3

signs = '.,?!:; '
numbers = ''.join([str(i) for i in range(10)])
RUSSIAN_LETTERS_AND_SIGNS = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя' + signs + numbers
ENGLISH_LETTERS_AND_SIGNS = ''.join([chr(i) for i in range(ord('a'), ord('z') + 1)]) + signs + numbers
print(RUSSIAN_LETTERS_AND_SIGNS, ENGLISH_LETTERS_AND_SIGNS)

абвгдеёжзийклмнопрстуфхцчшщъыьэюя.,?!:; 0123456789 abcdefghijklmnopqrstuvwxyz.,?!:; 0123456789


In [ ]:
!wget https://www.dropbox.com/s/yy2zqh34dyhv07i/data.txt?dl=1 -O data.txt

--2025-04-14 16:35:53--  https://www.dropbox.com/s/yy2zqh34dyhv07i/data.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/mw8tdyetqboqwkn5ma886/data.txt?rlkey=t9fmsizx27ikh0vak0ir265a6&dl=1 [following]
--2025-04-14 16:35:53--  https://www.dropbox.com/scl/fi/mw8tdyetqboqwkn5ma886/data.txt?rlkey=t9fmsizx27ikh0vak0ir265a6&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc81afef190ecb815e86a2de348a.dl.dropboxusercontent.com/cd/0/inline/CnysX8j-ceV5dGbT0KckknL8Flblt3lK71Ozw2LTd6SQNcI8fuTt6aEbmiPtmE7zHAZBskMOm139uzO9W7nm3cSZyTvy9SCoozbWcxZJKLoDHQNnHFme1vZwUpfG-wkDWrY/file?dl=1# [following]
--2025-04-14 16:35:54--  https://uc81afef190ecb815e86a2de348a.dl.dropboxusercontent.com/cd/0/inline/CnysX8j-ceV

In [ ]:
# create model folder
if not os.path.exists(MODEL_FOLDER_PATH):
  os.makedirs(MODEL_FOLDER_PATH)

In [ ]:
# clean text, only availaible symbols and lower text
def clean_text(text):
    text = text.lower().strip()
    res_text = ''.join([
        tt for tt in text if tt in RUSSIAN_LETTERS_AND_SIGNS or tt in ENGLISH_LETTERS_AND_SIGNS
      ])
    return res_text

In [ ]:
# create our tokenizer is similar to different exist tokenizer
class Tokenizer:
  def __init__(self, texts, sos, eos, unk, pad, tokens=None):
    self.sos = sos
    self.eos = eos
    self.unk = unk
    self.pad = pad
    self.tokens = tokens

    self.create_tokenizer(texts)

  def create_tokenizer(self, texts):
    texts = [clean_text(text) for text in texts] # clean each text

    tokens = set('\n'.join(list(texts)).replace('\n', '')) # get tokens (symbols)
    tokens = [t for t in tokens if t not in (self.sos, self.eos, self.unk) and len(t)]
    self.tokens = self.tokens or [self.sos, self.eos, self.unk, self.pad] + tokens

    self.token2idx = {token: i for i, token in enumerate(self.tokens)}
    self.idx2token = {i: token for i, token in enumerate(self.tokens)}
    self.unk_idx = self.token2idx[self.unk]
    self.sos_idx = self.token2idx[self.sos]
    self.eos_idx = self.token2idx[self.eos]
    self.pad_idx = self.token2idx[self.pad]

  def tokenize(self, text):
    #converts text to a list of tokens
    tokens = []
    for tok in text:
      if tok in self.token2idx:
        tokens.append(tok)
      else:
        tokens.append(self.unk_idx)

    return [self.sos] + tokens + [self.eos]

  def convert_tokens_to_idx(self, tokens):
    # convert tokens list to idx list
    idx_list = [self.token2idx.get(tok, self.unk_idx) for tok in tokens]

    return idx_list

  def convert_text_to_idx(self, text, seq_len=None):
    # convert text to idx list
    tokens = self.tokenize(text)[:seq_len]

    return self.convert_tokens_to_idx(
        tokens
    )

  def convert_idx_to_tokens(self, idx_list):
    # convert idx list to tokens list
    ans = []
    for idx in idx_list:
      ans.append(self.idx2token[idx])

    return ans


In [ ]:
# create tokenizer based on train_texts_path
def prepare_tokenizer(texts_path):
    with open(texts_path) as f:
        lines = [clean_text(line.strip()) for line in f.readlines()]

    return Tokenizer(lines, sos=SOS_TOKEN, eos=EOS_TOKEN, unk=UNK_TOKEN, pad=PAD_TOKEN)

In [ ]:
# read, shuffle, split and reorganise data
with open(FULL_DATA_PATH) as f:
  split_lines = [line.strip().split('\t') for line in f.readlines()]

en_texts = [line[0] for line in split_lines]
ru_texts = [line[1] for line in split_lines]

en_train_texts, en_val_texts, ru_train_texts, ru_val_texts = train_test_split(
    en_texts,
    ru_texts,
    test_size=VALIDATION_PROP,
    random_state=RANDOM_STATE,
    shuffle=True
  )

# filter data by russian text length
with open(X_TRAIN_PATH, 'w') as f:
  f.write('\n'.join([tt for i, tt in enumerate(en_train_texts) if len(ru_train_texts[i]) < MAX_TEXT_LEN]))

with open(Y_TRAIN_PATH, 'w') as f:
  f.write('\n'.join([tt for i, tt in enumerate(ru_train_texts) if len(tt) < MAX_TEXT_LEN]))

with open(X_VALID_PATH, 'w') as f:
  f.write('\n'.join([tt for i, tt in enumerate(en_val_texts) if len(ru_val_texts[i]) < MAX_TEXT_LEN]))

with open(Y_VALID_PATH, 'w') as f:
  f.write('\n'.join([tt for i, tt in enumerate(ru_val_texts) if len(tt) < MAX_TEXT_LEN]))


In [ ]:
# create tokenizers
x_tokenizer = prepare_tokenizer(X_TRAIN_PATH)
y_tokenizer = prepare_tokenizer(Y_TRAIN_PATH)

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self,
                 x_seq_path,
                 x_tokenizer=None,
                 x_seq_len=None,
                 is_train=True,
                 y_seq_path=None,
                 y_tokenizer=None,
                 y_seq_len=None):
        self.is_train = is_train
        with open(x_seq_path) as f:
          self.x_seq_list = [line.strip() for line in f.readlines()]

        self.x_tokenizer = x_tokenizer or prepare_tokenizer(x_seq_path)
        self.x_seq_len = x_seq_len

        if self.is_train:
          with open(y_seq_path) as f:
            self.y_seq_list = [line.strip() for line in f.readlines()]
          self.y_tokenizer = y_tokenizer or prepare_tokenizer(y_seq_path)
        else:
          self.y_seq_list = None
          self.y_tokenizer = None
        self.y_seq_len = y_seq_len

    def __getitem__(self, idx):
        # pad all sequences to fixed len (for x length equals x_seq_len, for y length equals y_seq_len)
        x = clean_text(self.x_seq_list[idx])
        x = self.x_tokenizer.convert_text_to_idx(x, seq_len=self.x_seq_len)
        x_pad_part = [self.x_tokenizer.pad_idx] * max(0, self.x_seq_len - len(x))
        x = x + x_pad_part

        x = torch.tensor(x)
        if self.is_train:
            y = clean_text(self.y_seq_list[idx])
            y = self.y_tokenizer.convert_text_to_idx(y, seq_len=self.y_seq_len)[1:]
            y_pad_part = [self.y_tokenizer.pad_idx] * max(0, self.y_seq_len - len(y))
            y = y + y_pad_part
            y = torch.tensor(y)

            return x, y
        return x

    def cnt_y_tokens(self):
      return len(self.y_tokenizer.tokens)

    def cnt_x_tokens(self):
      return len(self.x_tokenizer.tokens)

    def __len__(self):
        return len(self.x_seq_list)

In [ ]:
class EncoderDecoderModel(nn.Module):
    # Simple Encoder-Decoder Model with 3 GRU Layers in encoder
    # And with 3 GRU in decoder, forward could be run with teacher_forcing
    def __init__(self,
                 input_voc_size,
                 cnt_classes,
                 device=None,
                 input_size=INPUT_SIZE,
                 hidden_size=HIDDEN_SIZE,
                 lstm_layers=LSTM_LAYERS,
                 is_bidirectional=IS_BIDIRECTIONAL_LAYERS):
        super(EncoderDecoderModel, self).__init__()

        self.input_size = input_size
        self.lstm_layers = lstm_layers
        self.cnt_classes = cnt_classes
        self.hidden_size = hidden_size
        self.out_features = 1 if self.cnt_classes == 2 else self.cnt_classes
        self.is_bidirectional = is_bidirectional

        self.encoder_embed = nn.Embedding(input_voc_size, self.input_size)
        self.encoder_lstm_layer = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=self.lstm_layers,
            bidirectional=self.is_bidirectional,
            batch_first=True
        )

        self.decoder_embed = nn.Embedding(self.cnt_classes, self.input_size)
        self.decoder_lstm_layer = nn.LSTM(
            input_size=self.hidden_size,
            hidden_size=(1 + self.is_bidirectional) * self.hidden_size,
            num_layers=self.lstm_layers,
            batch_first=True
        )
        self.decoder_linear = nn.Linear(
            in_features=self.hidden_size,
            out_features=self.out_features)

    def forward(
            self,
            x,
            target_val,
            teacher_forcing_ratio = TEACHER_FORCING_RATIO):
        batch_n = x.shape[0]
        length_tensor = (x != PAD_IND).sum(axis=1).cpu() # get seq lens to pack

        # encoder
        x = self.encoder_embed(x) #[bs, len_seq, 768]
        x = pack_padded_sequence(x, length_tensor, batch_first=True, enforce_sorted=False)

        _, prev_state = self.encoder_lstm_layer(x)

        prev_state = (prev_state[0].reshape(self.lstm_layers, batch_n, -1),
                      prev_state[1].reshape(self.lstm_layers, batch_n, -1))

        prev_token_idx = torch.tensor([SOS_IND] * batch_n).reshape(-1, 1).to(device)

        probs_tensor = None
        for i in range(NEC_TOKENS_LEN):
            if i != 0 and target_val is not None and random.random() < teacher_forcing_ratio:
                prev_token_idx = target_val[:, i - 1].reshape(-1, 1)

            prev_token_emb = self.decoder_embed(prev_token_idx)
            out, prev_state = self.decoder_lstm_layer(prev_token_emb, prev_state)

            out = self.decoder_linear(out)

            # token idx answer by greedy strategy
            prev_token_idx = out.argmax(axis=-1).reshape(batch_n, -1)

            # save probs for answers
            if probs_tensor is None:
                probs_tensor = out.unsqueeze(0)
            else:
                probs_tensor = torch.cat([probs_tensor, out.unsqueeze(0)])

            #if prev_token_idx.unique().shape[0] == 1 and prev_token_idx[0] == EOS_IND:
              #break

        return probs_tensor.transpose(1, 0)

In [ ]:
# create train and val dataset (val tokenizers are the same as in train)
train_dataset = TranslationDataset(
    x_seq_path=X_TRAIN_PATH,
    y_seq_path=Y_TRAIN_PATH,
    is_train=True,
    x_seq_len=NEC_TOKENS_LEN,
    y_seq_len=NEC_TOKENS_LEN)
val_dataset = TranslationDataset(
    x_seq_path=X_VALID_PATH,
    x_tokenizer=train_dataset.x_tokenizer,
    y_seq_path=Y_VALID_PATH,
    x_seq_len=NEC_TOKENS_LEN,
    y_seq_len=NEC_TOKENS_LEN,
    y_tokenizer=train_dataset.y_tokenizer,
    is_train=True
)

# create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [ ]:
# cnt classes if it is neccessary
if CNT_CLASSES is None:
  CNT_CLASSES = train_dataset.cnt_y_tokens()

In [ ]:
# define model and move it to device
model = EncoderDecoderModel(
    input_voc_size=train_dataset.cnt_x_tokens(),
    device=device,
    cnt_classes=train_dataset.cnt_y_tokens())
model = model.to(device)

In [ ]:
# use CrossEntropyLoss because the task is something like classification
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=train_dataset.x_tokenizer.pad_idx)
# define optimizer
optimizer = Adam(model.parameters(), lr=LR)


# define scheduler (policy of lr change)
lambda_scheduler = lambda x: SCHEDULER_LAMBDA_PARAM ** x
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_scheduler)



In [ ]:
def show_random_examples(input_data, out_answers, targets, dataloader):
    # show random texts with three ids
    for i in [1, 3, -1]:
        x_data = input_data[i]
        y_data = out_answers[i]
        target_data = targets[i]

        # get tokenizers
        x_tokenizer = dataloader.dataset.x_tokenizer
        y_tokenizer = dataloader.dataset.y_tokenizer

        # convert answers idx
        x_answers = x_tokenizer.convert_idx_to_tokens(x_data.cpu().detach().numpy().tolist())
        y_answers = y_tokenizer.convert_idx_to_tokens(y_data.cpu().detach().numpy().tolist())
        target_tokens = y_tokenizer.convert_idx_to_tokens(target_data.cpu().detach().numpy().tolist())

        print("X: ", ''.join([x for x in x_answers]))
        print("Answer: ", ''.join([y for y in y_answers]))
        print("Target: ", ''.join([trg for trg in target_tokens]))


def correct_tokens_for_bleu(tokens_list):
    res_tokens_list = []
    for tok in tokens_list:
      if tok == EOS_TOKEN:
        break
      res_tokens_list.append(tok)

    return [res_tokens_list]


def bleu_calculation(targets_ids, all_labels, tokenizer, bleu_metric):
    targets_ids = np.array(list(itertools.chain(*targets_ids))) # concat list by axis=0
    all_labels = np.array(list(itertools.chain(*all_labels)))

    targets_ids = targets_ids.reshape(-1, targets_ids.shape[-1])
    all_labels = all_labels.reshape(-1, all_labels.shape[-1])

    targets_list = [
        correct_tokens_for_bleu(
            tokenizer.convert_idx_to_tokens(trg_idx_list)
          )[0] for trg_idx_list in targets_ids]
    answers_list = [correct_tokens_for_bleu(
          tokenizer.convert_idx_to_tokens(label_list)
        ) for label_list in all_labels]

    return bleu_metric.compute(predictions=targets_list, references=answers_list)['bleu']


def train_one_epoch(
      epoch_index,
      training_loader,
      scheduler,
      optimizer,
      loss_fn,
      clip,
      teacher_forcing_ratio=TEACHER_FORCING_RATIO):
    # train one epoch with scheduler, writer and clip
    running_loss = 0.
    last_loss = 0.
    train_loss = 0.
    i = 0

    all_labels = []
    all_answers = []

    for i, data in tqdm(enumerate(training_loader)):
        input_data, labels = data

        optimizer.zero_grad()
        # move data to device
        input_data = input_data.to(device)
        labels = labels.to(device).long()

        # model inference with teacher_forcing
        outputs = model(input_data, labels, teacher_forcing_ratio=teacher_forcing_ratio)

        answers = outputs.argmax(axis=-1).squeeze(-1)

        loss = loss_fn(outputs.reshape(-1, CNT_CLASSES), labels[:, :outputs.shape[1]].reshape(-1))

        # backward
        loss.backward()
        # grad normalization
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        running_loss += loss.item()
        train_loss += loss.item()

        all_answers += [answers.cpu().detach().numpy().tolist()]
        all_labels += [labels.cpu().detach().numpy().tolist()]
        if i % WRITE_CNT_STEPS == WRITE_CNT_STEPS - 1 and epoch_index != -1:
            print("i =", i)
            # get answers idx by argmax
            show_random_examples(input_data, answers, labels, train_dataloader)

            # logging
            last_loss = running_loss / 100 # loss per batch
            tb_x = epoch_index * len(training_loader) + i + 1
            running_loss = 0.

            print("TRAIN LOSS =", last_loss)
            if i % SCHEDULER_CNT_STEPS == 0 and scheduler is not None:
                scheduler.step()

        i += 1
    bleu_score = bleu_calculation(
        all_answers,
        all_labels,
        training_loader.dataset.y_tokenizer,
        bleu_metric)
    print("TRAIN BLEU:", bleu_score)
    train_loss = train_loss / len(training_loader)

    return last_loss, bleu_score


def validation(epoch_number, val_dataloader):
    val_loss = 0.0
    all_answers = []
    all_labels = []

    # eval model mode
    model.eval()
    with torch.no_grad():
        for i, vdata in tqdm(enumerate(val_dataloader)):
            input_data, labels = vdata

            optimizer.zero_grad()

            # move data to device
            input_data = input_data.to(device)
            labels = labels.to(device).long()

            # model inference
            outputs = model(input_data, None)

            outputs_reshape = outputs.reshape(-1, CNT_CLASSES)

            # calc loss
            vloss = loss_fn(outputs_reshape, labels[:, :outputs.shape[1]].reshape(-1))
            val_loss += vloss.item()


            answers = outputs.argmax(axis=-1).squeeze(-1)
            all_answers += [answers.cpu().detach().numpy().tolist()]
            all_labels += [labels.cpu().detach().numpy().tolist()]
            # logging
            if i == 0:
                show_random_examples(input_data, answers, labels, val_dataloader)

    val_loss = val_loss / len(val_dataloader)

    bleu_score = bleu_calculation(
        all_answers,
        all_labels,
        val_dataloader.dataset.y_tokenizer,
        bleu_metric)

    print("VAL LOSS =", val_loss)
    return val_loss, bleu_score

In [ ]:
# train
def train(
    model,
    model_name,
    train_dataloader,
    optimizer,
    scheduler, # could be None
    loss_fn,
    clip,
    teacher_forcing_ratio,
    epochs):
  best_vloss = 1_000_000.

  for epoch_number in tqdm(range(epochs)):
      print('EPOCH {}:'.format(epoch_number + 1))

      model.train(True)
      train_loss, train_bleu = train_one_epoch(
          epoch_index=epoch_number,
          training_loader=train_dataloader,
          optimizer=optimizer,
          scheduler=scheduler,
          loss_fn=loss_fn,
          clip=clip,
          teacher_forcing_ratio=teacher_forcing_ratio)

      model.train(False)
      val_loss, val_bleu = validation(epoch_number, val_dataloader)

      # # logging
      # writer.add_scalar('Bleu/train', train_bleu, epoch_number)
      # writer.add_scalar('Loss/valid', val_loss, epoch_number)
      # writer.add_scalar('Bleu/valid', val_bleu, epoch_number)

      # save models if val_loss is better than best_vloss
      if val_loss < best_vloss:
          best_vloss = val_loss
          model_path = os.path.join(model_name, 'model_{}'.format(epoch_number + 1,))
          torch.save(model.state_dict(), model_path)

  return model


In [ ]:
base_enc_dec_model = train(
    model=model,
    model_name=MODEL_NAME,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    scheduler=scheduler,
    loss_fn=loss_fn,
    clip=CLIP,
    teacher_forcing_ratio=TEACHER_FORCING_RATIO,
    epochs=EPOCHS
)

  0%|          | 0/10 [00:00<?, ?it/s]

EPOCH 1:


0it [00:00, ?it/s]

i = 99
X:  <SOS>towels and bed linen are featured in this villa.<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
Answer:  в                                                                                                   
Target:  предоставляются полотенца и постельное белье.<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
X:  <SOS>the property has a landscaped garden and a fenced garden with garden furniture.<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
Answer:  в                                         

KeyboardInterrupt: 

# Teacher Forcing

In [ ]:
MODEL_NAME = 'lstm_teacher_forcing_model'
TEACHER_FORCING_RATIO = 0.5

if not os.path.exists(MODEL_NAME):
  os.makedirs(MODEL_NAME)

# define model and move it to device
model = EncoderDecoderModel(
    input_voc_size=train_dataset.cnt_x_tokens(),
    device=device,
    cnt_classes=train_dataset.cnt_y_tokens())
model = model.to(device)

# writer

# redefine optimizer
optimizer = Adam(model.parameters(), lr=LR)

In [ ]:
teacher_forcing_enc_dec_model = train(
      model=model,
      model_name=MODEL_NAME,
      train_dataloader=train_dataloader,
      optimizer=optimizer,
      scheduler=scheduler,
      loss_fn=loss_fn,
      clip=CLIP,
      teacher_forcing_ratio=TEACHER_FORCING_RATIO,
      epochs=EPOCHS
  )

  0%|          | 0/10 [00:00<?, ?it/s]

EPOCH 1:


0it [00:00, ?it/s]

i = 99
X:  <SOS>you will find a 24hour front desk and a garden at the property.<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
Answer:  в   о                                                                                               
Target:  стойка регистрации работает круглосуточно. на территории гостевого дома разбит сад.<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
X:  <SOS>some rooms feature views of the sea or city.<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
Answer:  в   о                                                                                               
Target:  из н

ValueError: Predictions and/or references don't match the expected format.
Expected format:
Feature option 0: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}
Feature option 1: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')},
Input predictions: ['э', 'э', 'э', ..., 'э', 'э', 'э'],
Input references: [['в', ' ', 'р', 'е', 'с', 'т', 'о', 'р', 'а', 'н', 'е', ' ', 'п', 'о', 'д', 'а', 'ю', 'т', ' ', 'т', 'р', 'а', 'д', 'и', 'ц', 'и', 'о', 'н', 'н', 'ы', 'е', ' ', 'и', 'з', 'ы', 'с', 'к', 'а', 'н', 'н', 'ы', 'е', ' ', 'б', 'л', 'ю', 'д', 'а', ' ', 'р', 'е', 'г', 'и', 'о', 'н', 'а', ' ', 'а', 'с', 'т', 'у', 'р', 'и', 'я', ',', ' ', 'п', 'р', 'и', 'г', 'о', 'т', 'о', 'в', 'л', 'е', 'н', 'н', 'ы', 'е', ' ', 'н', 'а', ' ', 'д', 'р', 'о', 'в', 'я', 'н', 'о', 'м', ' ', 'г', 'р', 'и', 'л', 'е', '.', '<PAD>']]